In [3]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az

# Read the data with error handling
data = pd.read_csv('BostonHousing.csv', na_values=[''], on_bad_lines='skip')

# Drop any rows with missing values
data = data.dropna()

# Standardize the predictors
X = data[['rm', 'crim', 'indus']]
X_standardized = (X - X.mean()) / X.std()
y = data['medv']

# Create and fit the PyMC model
with pm.Model() as model:
    # Priors for unknown model parameters
    intercept = pm.Normal('intercept', mu=0, sigma=10)
    betas = pm.Normal('betas', mu=0, sigma=2, shape=3, dims='predictors')
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Expected value of outcome
    mu = intercept + pm.math.dot(X_standardized, betas)

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=y)

    # Inference
    trace = pm.sample(2000, tune=2000, return_inferencedata=True, target_accept=0.95)

# Print summary of the model parameters
summary = az.summary(trace, var_names=['intercept', 'betas', 'sigma'])
print("\nModel Parameter Summary:")
print(summary)

# Calculate 95% HDI for parameters
hdi = az.hdi(trace, hdi_prob=0.95)
print("\n95% HDI for parameters:")
print(hdi)

# Get the variable importance by looking at the absolute values of the standardized coefficients
var_importance = pd.DataFrame({
    'Variable': ['rm', 'crim', 'indus'],
    'Mean Effect': az.summary(trace, var_names=['betas'])['mean'].values
})
var_importance['Abs Effect'] = abs(var_importance['Mean Effect'])
var_importance = var_importance.sort_values('Abs Effect', ascending=False)
print("\nVariable Importance:")
print(var_importance)

# Posterior predictive simulation
with model:
    post_pred = pm.sample_posterior_predictive(trace)

# Calculate 50% prediction interval
y_pred = post_pred.posterior_predictive.Y_obs
pred_intervals = np.percentile(y_pred, [25, 75], axis=1)
print("\n50% Prediction Interval:")
print("Lower bound (25th percentile):", pred_intervals[0].mean())
print("Upper bound (75th percentile):", pred_intervals[1].mean())

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, betas, sigma]


Output()

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 45 seconds.



Model Parameter Summary:
             mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
intercept  22.525  0.260  22.035   23.007      0.002    0.002   12118.0   
betas[0]    5.286  0.286   4.749    5.828      0.003    0.002    9428.0   
betas[1]   -1.716  0.289  -2.262   -1.182      0.003    0.002    8457.0   
betas[2]   -1.659  0.305  -2.242   -1.093      0.004    0.003    7430.0   
sigma       5.905  0.177   5.584    6.242      0.002    0.001    8872.0   

           ess_tail  r_hat  
intercept    6224.0    1.0  
betas[0]     5996.0    1.0  
betas[1]     6302.0    1.0  
betas[2]     6199.0    1.0  
sigma        6220.0    1.0  

95% HDI for parameters:
<xarray.Dataset>
Dimensions:     (hdi: 2, predictors: 3)
Coordinates:
  * predictors  (predictors) int32 0 1 2
  * hdi         (hdi) <U6 'lower' 'higher'
Data variables:
    intercept   (hdi) float64 22.01 23.02
    betas       (predictors, hdi) float64 4.705 5.828 -2.271 ... -2.276 -1.081
    sigma       (hdi) float64 5.5

Sampling: [Y_obs]


Output()


50% Prediction Interval:
Lower bound (25th percentile): 18.530485274084125
Upper bound (75th percentile): 26.52247048406406
